In [ ]:
import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
import shap
import pandas as pd
import openai
from IPython.display import display

In [ ]:
openai.api_base = os.getenv('OPENROUTER_API_BASE', 'https://openrouter.ai/api/v1')
openai.api_key  = os.getenv('OPENROUTER_API_KEY')

In [ ]:
def extract_frames(video_path: str, interval: int = 30):
    """Yield (frame_index, PIL.Image) at every `interval` frames."""
    cap = cv2.VideoCapture(video_path)
    idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % interval == 0:
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            yield idx, Image.fromarray(img)
        idx += 1
    cap.release()

In [ ]:
# Load model and explainer once
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('pytorch/vision', 'vit_b_16', pretrained=True).to(device)
model.eval()

preprocess = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
])
background = torch.zeros((1, 3, 224, 224), device=device)
explainer = shap.GradientExplainer(model, background)

In [ ]:
def compute_shap_for_video(video_path: str, interval: int = 30):
    indices, shap_vals = [], []
    for idx, frame in extract_frames(video_path, interval):
        inp = preprocess(frame).unsqueeze(0).to(device)
        shap_val = explainer.shap_values(inp)
        indices.append(idx)
        shap_vals.append(np.array(shap_val))
    return np.array(indices), np.stack(shap_vals)

In [ ]:
video_file = 'path/to/video.mp4'
indices, shap_vals = compute_shap_for_video(video_file, interval=30)
print('Extracted frames:', indices)

In [ ]:
def summarize_shap(shap_vals: np.ndarray, top_k: int = 3) -> str:
    """Return a brief summary of the most influential frames."""
    stats = np.mean(np.abs(shap_vals), axis=(1,2,3))
    top_idxs = np.argsort(stats)[-top_k:][::-1]
    means = stats[top_idxs].round(3).tolist()
    return f"Top frames: {top_idxs.tolist()}, mean abs SHAP: {means}"

In [ ]:
def generate_description(shap_summary: str, tag: str) -> str:
    prompt = (
        f"Video classified as {tag}. SHAP summary: {shap_summary}. "
        "Explain in 2-3 sentences why the model considers it real or fake."
    )
    resp = openai.ChatCompletion.create(
        model='deepseek-v3',
        messages=[{'role':'user','content':prompt}],
        temperature=0.7,
        max_tokens=150
    )
    return resp.choices[0].message.content.strip()

In [ ]:
summary = summarize_shap(shap_vals)
description = generate_description(summary, 'fake')
print(description)

In [ ]:
# CSV mapping: columns ['video_filename','tag']
tag_df = pd.read_csv('video_tags.csv')
results = []

for _, row in tag_df.iterrows():
    path, tag = row['video_filename'], row['tag']
    idxs, vals = compute_shap_for_video(path)
    summ = summarize_shap(vals)
    desc = generate_description(summ, tag)
    results.append({'video_path': path, 'tag': tag, 'description': desc})

df = pd.DataFrame(results)

display(df)                # show in notebook
df.to_excel('results.xlsx', index=False)  
print('Saved results.xlsx')